# Fine Tuning

In [ ]:
import numpy as np
import pandas as pd
import os
import torch
import torchvision
import torch.nn as nn
import torch.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,Dataset
from torchvision import datasets,models,transforms
from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit
torch.version

In [ ]:
os.listdir("../input")

## 注意事项

1.新数据集和原始数据集类似，那么直接可以微调最后一个FC层或者重新指定一个分类器；<p>
2.如果差异比较大，可以从模型的中部开始训练，只对最后几层进行fine_tuning；<p>
3.如果差异比较大，并且上面的方法不可行，最好重新训练模型，只将预训练的模型作为新模型的一个初始化数据；<p>
4.新数据集的大小一定要与原始数据集相同<p>
5.如果数据集大小不相同的话，可以在最后的fc层之前添加卷积或者pool层，但是这样做会使得精度大幅度下降；<p>
6.对于不同的层可以设置不同的学习率；<p>

## 微调实例

In [ ]:
all_labels_df=pd.read_csv("../input/labels.csv")
all_labels_df.head()

定义两个字典

In [ ]:
breeds=all_labels_df.breed.unique()
breed2idx=dict((breed,idx) for idx,breed in enumerate(breeds))
idx2breed=dict((idx,breed) for idx,breed in enumerate(breeds))
print(len(breeds))   # 这个数据有120个类别

把类别标号添加到原始表格中

In [ ]:
all_labels_df["label_idx"]=[breed2idx[b] for b in all_labels_df.breed]
all_labels_df.head()

由于我们的数据集是官方指定的格式，所以我们自己定义一个数据集

In [ ]:
class DogDataset(Dataset):
    def __init__(self,label_df,img_path,transform=None):
        self.label_df=label_df
        self.img_path=img_path
        self.transform=transform
        
    def __len__(self):
        """放回数据集的长度"""
        return self.label_df.shape[0]
    
    def __getitem__(self,idx):
        """读取图片和标签"""
        label=self.label_df.label_idx[idx]
        id_img=self.label_df.id[idx]
        img_P=os.path.join(self.img_path,id_img)+".jpg"
        img=Image.open(img_P)
        
        if self.transform:
            img=self.transform(img)
            
        return img,label

定义一些超参数

In [ ]:
IMG_SIZE=224   # resnet50的输入是224的，所以需要将图片统一大小
BATCH_SIZE=256   # 每个批次输入的图片的数量
IMG_MEAN=[0.485,0.456,0.406]
IMG_STD=[0.229,0.224,0.225]
CUDA=torch.cuda.is_available()
DEVICE=torch.device("cuda" if CUDA else "cpu")

定义数据和图片变换规则

In [ ]:
train_transform=transforms.Compose([
    transforms.Resize(IMG_SIZE),   # 改变图片大小
    transforms.RandomResizedCrop(IMG_SIZE),  # 首先随机裁减，然后在转换成规定图片大小
    transforms.RandomHorizontalFlip(),  # 以0.5的概率随机水平翻转
    transforms.RandomRotation(30),  # 旋转
    transforms.ToTensor(),  # 转换成张量
    transforms.Normalize(IMG_MEAN,IMG_STD)  # 标准化
])

val_transform=transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(IMG_MEAN,IMG_STD)
])

我们这里切割10%的数据作为训练时的验证数据

In [ ]:
# 使用分层抽样切分训练集和验证集
dataset_name=["train","valid"]
stra_splt=StratifiedShuffleSplit(n_splits=1,test_size=0.1,random_state=0)
train_index,val_index=next(iter(stra_splt.split(all_labels_df.id,all_labels_df.breed)))
train_df=all_labels_df.iloc[train_index,:].reset_index()
val_df=all_labels_df.iloc[val_index,:].reset_index()
print(len(train_df))
print(len(val_df))

使用官方的dataloader载入数据

In [ ]:
image_tarnsforms={"train":train_transform,"valid":val_transform}

train_dataset=DogDataset(train_df,"../input/train",transform=image_tarnsforms["train"])
valid_dataset=DogDataset(val_df,"../input/train",transform=image_tarnsforms["valid"])
image_dataset={"train":train_dataset,"valid":valid_dataset}

image_dataloader={x:DataLoader(image_dataset[x],batch_size=BATCH_SIZE,shuffle=True,num_workers=0) for x in dataset_name}
datasize={x:len(image_dataset[x]) for x in dataset_name}

配置我们的网络，由于ImageNet识别的是1000个类别，而我们的数据集只有100个类别。

In [ ]:
model_ft=models.resnet50(pretrained=True) # 自动下载官方的预训练模型
# 将所有的层都先冻结
for param in model_ft.parameters():
    param.requires_grad=False

# 打印全连接层的信息
print(model_ft.fc)
num_fc_ftr=model_ft.fc.in_features  # 获取全连接层的输入
model_ft.fc=nn.Linear(num_fc_ftr,len(breeds))  # 定义一个新的全连接层
model_ft=model_ft.to(DEVICE)  # 将网络放到设备中
print(model_ft)  # 最后打印一下模型

设置训练参数

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam([{"params":model_ft.fc.parameters()}],lr=0.001)  # 指定新加的fc层的学习率

In [ ]:
for data in image_dataloader["train"].dataset:
    x,y=data
    plt.imshow(x)
    plt.title(y)
    break

定义训练函数

In [ ]:
def train(model,device,train_loader,epoch):
    model.train()
    for batch_idx,data in enumerate(train_loader):
        x,y=data
        x=x.to(device)
        y=y.to(device)
        optimizer.zero_grad()
        y_hat=model(x)
        loss=criterion(y_hat,y)
        loss.backward()
        optimizer.step()
    print ('Train Epoch: {}\t Loss: {:.6f}'.format(epoch,loss.item()))

定义测试函数

In [ ]:
def test(model,device,test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for i,data in enumerate(test_loader):
            x,y=data
            x=x.to(device)
            y=y.to(device)
            optimizer.zero_grad()
            y_hat=model(x)
            test_loss+=criterion(y_hat,y).item()
            pred=y_hat.max(1,keepdim=True)[1]
            correct+=pred.eq(y.view_as(pred)).sum().item()
    test_loss/=len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_dataset),
        100. * correct / len(valid_dataset)))

训练9次，看看效果

In [ ]:
for epoch in range(1,9):
    %time train(model_ft,DEVICE,image_dataloader["train"],epoch)
    test(model_ft,DEVICE,image_dataloader["valid"])

我们每次都需要将一张图片在全部网络中进行计算，而且每次结果都是一样的，这样浪费了很多计算资源。下面我们将这些不进行反向传播或者说不更新参数层的计算结果保存下来，这样我们以后使用的时候就可以直接将这些结果输入到以这些结果构建的新的网络层中，省去了计算时间，这样如果只训练全连接层，cpu就能完成了。

## 固定层的向量导出

In [94]:
# hook
in_list=[]  # 存放所有的输出
def hook(module,input,output):
    for i in range(input[0].size(0)):
        in_list.append(input[0][i].cpu().numpy())

In [95]:
model_ft.avgpool.register_forward_hook(hook)

In [98]:
%%time
with torch.no_grad():
    for batch_idx,data in enumerate(image_dataloader["train"]):
        x,y=data
        x=x.to(DEVICE)
        y=y.to(DEVICE)
        y_hat=model_ft(x)

CPU times: user 1min 9s, sys: 16.6 s, total: 1min 26s
Wall time: 1min 31s


In [103]:
features=np.array(in_list)

MemoryError: 

In [105]:
np.save("/kaggle/working/features",features)

In [107]:
features.shape

(9199, 2048, 7, 7)

这样，我们就只需要将这个数据读出来，再输入到我们之后的分类器中就可以，或者更高级的分类器，如SVM中。